# Import Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import MDS
import pandas as pd
import scipy.io
import pickle
from matplotlib.pyplot import figure
from IPython.core.debugger import set_trace
from sklearn.preprocessing import scale
from scipy.special import expit
from scipy.stats import norm
import json
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
pio.renderers.default = 'iframe'

# Load experimental data

In [2]:
APPNAME = 'shepdyads' #shepdyadscls or shepdyads
FILTER_EXPERIENCE = True
EXPERIENCE_THRESHOLD = 0
NBOOT = 1000
PATH = f"data/{APPNAME}-data/psynet/data/"
nodes = pd.read_csv(PATH + "node.csv", low_memory=False)
infos = pd.read_csv(PATH + "info.csv", low_memory=False)
networks = pd.read_csv(PATH + "network.csv", low_memory=False)
participants = pd.read_csv(PATH + "participant.csv", low_memory=False)
questions = pd.read_csv(PATH + "response.csv", low_memory=False)

MIN_INTERVAL = 60
MAX_INTERVAL = 71
list_of_chords = [json.dumps([i,j]) for i in range(MIN_INTERVAL,MAX_INTERVAL+1) for j in range(i,MAX_INTERVAL+1)]


# filter networks
network_data = networks
network_data = network_data[network_data["role"] == "experiment"]
network_data = network_data[network_data["failed"] == 'f']
network_data = network_data[network_data["trial_maker_id"] == 'main_experiment']

experiment_net_id = list(network_data['id'].to_numpy())

# filter participants
valid_participants = participants
valid_participants = valid_participants[valid_participants["complete"] == "t"]

# filter based on musical expertise
if FILTER_EXPERIENCE:
    musical_experience = questions[questions["question"] == "years_playing_music"]
    musical_experience = musical_experience[['participant_id','answer']]
    musical_experience = musical_experience.assign(
        numeric_response = lambda dataframe: dataframe['answer'].map(lambda answer: json.loads(answer))
    )
    musical_experience = musical_experience[pd.to_numeric(musical_experience['numeric_response'], errors='coerce').notnull()]
    musical_experience['numeric_response'] = musical_experience['numeric_response'].astype(float) #int
    musical_experience = musical_experience[musical_experience['numeric_response'] >= EXPERIENCE_THRESHOLD]
    experienced_ids = list(musical_experience["participant_id"].to_numpy())
    valid_participants = valid_participants[valid_participants["id"].isin(experienced_ids)]

valid_participant_id = list(valid_participants['id'].to_numpy())

# filter trials
trial_data = infos
trial_data = trial_data[trial_data["type"] == "custom_trial"]
trial_data = trial_data[trial_data["failed"] == "f"]
trial_data = trial_data[trial_data["complete"] == "t"]
trial_data = trial_data[trial_data["is_repeat_trial"] == "f"]
trial_data = trial_data[trial_data["network_id"].isin(experiment_net_id)]
trial_data = trial_data[trial_data["participant_id"].isin(valid_participant_id)]
trial_data = trial_data[["id", "origin_id", "response_id", "network_id", "participant_id", "definition", "answer"]]

# create response summary

def extract_from_json(definition, key):
    definition = json.loads(definition)
    return definition[key]

origin_id = trial_data["origin_id"].to_numpy()
participant_id = trial_data["participant_id"].to_numpy()
response_id = trial_data["response_id"].to_numpy()
definition = trial_data["definition"].to_numpy()
definition = [json.loads(el)["chords"] for el in trial_data["definition"].to_numpy()]
answer = trial_data["answer"].to_numpy()
answer = [json.loads(ans) for ans in answer]

synth = [el["synth"] for el in definition]
chord_1 = [el["dyad_class_1"] for el in definition]
chord_2 = [el["dyad_class_2"] for el in definition]
idx_1 = [el["idx_1"] for el in definition]
idx_2 = [el["idx_2"] for el in definition]

for i in range(len(chord_1)):
    c_1 = chord_1[i]
    c_2 = chord_2[i]
    i_1 = idx_1[i]
    i_2 = idx_2[i]
    if i_1 > i_2:
        chord_1[i] = c_2
        chord_2[i] = c_1
        idx_1[i] = i_2
        idx_2[i] = i_1

data = pd.DataFrame({
    "origin_id": origin_id,
    "participant_id": participant_id,
    "response_id": response_id,
    "answer": answer,
    "synth": synth,
    "chord_1": chord_1,
    "chord_2": chord_2,
    "idx_1": idx_1,
    "idx_2": idx_2
})


FileNotFoundError: [Errno 2] No such file or directory: 'data/shepdyads-data/psynet/data/node.csv'

In [ ]:
data

# Costs and demograhics

In [ ]:
participants = pd.read_csv(PATH + "participant.csv", low_memory=False)
valid_participants = participants[participants["complete"] == "t"]
experiment_summary = {
    "N_participants": valid_participants.shape[0],
    "cost": participants["base_pay"].sum() + participants["bonus"].sum()
}

experiment_summary


# Export to CSV

In [ ]:
data.to_csv("data_dyads.csv")

# Preprocess data

In [ ]:
Z_SCORE = True

def preprocess_data(data, list_of_chords, z_score):
    N = len(list_of_chords) # 81
    sim_mat = np.zeros(shape=(N,N))

    if not z_score:
        processed = data[["idx_1","idx_2","answer"]]
        processed = processed.groupby(["idx_1","idx_2"],as_index=False).mean()

        for i in range(len(processed.index)):
            idx1 = processed["idx_1"][i]
            idx2 = processed["idx_2"][i]
            sim_mat[idx1, idx2] = processed["answer"][i]

        sim_mat = sim_mat / 6
        sim_mat = sim_mat + np.transpose(sim_mat)
        np.fill_diagonal(sim_mat,1)

    else:
        processed = data[["idx_1","idx_2","participant_id","answer"]]
        processed = processed.dropna()
        gb = processed.groupby('participant_id') 
        splitted_data = [gb.get_group(x) for x in gb.groups]
        splitted_data_z = []
        for df in splitted_data:
            z_data = scale(df['answer'].to_numpy())
            updated_df = df.assign(z_answer = z_data)
            splitted_data_z = splitted_data_z + [updated_df]
        processed = pd.concat(splitted_data_z)
        processed = processed.groupby(["idx_1","idx_2"],as_index=False).mean()

        for i in range(len(processed.index)):
            idx1 = processed["idx_1"][i]
            idx2 = processed["idx_2"][i]
            sim_mat[idx1, idx2] = processed["z_answer"][i]

        sim_mat = norm.cdf(sim_mat + np.transpose(sim_mat))
        # sim_mat = sim_mat + np.transpose(sim_mat)
        np.fill_diagonal(sim_mat,1)


    experimental_data = {"similarity": sim_mat}
    
    return experimental_data

experimental_data = preprocess_data(data, list_of_chords, Z_SCORE)

In [ ]:
fig, axs = plt.subplots(1, 1)
fig.set_figheight(10)
fig.set_figwidth(10)

count = 0

for key in experimental_data.keys():
    idxs = count
    axs.matshow(experimental_data[key])
    axs.set_title(key, fontsize=20)
    axs.xaxis.set_visible(False)
    axs.yaxis.set_visible(False)
    count = count + 1

# Compute MDS embeddings

In [ ]:
transformed_data = {}
n_components = 3
np.random.seed(1233) #1233

for key in experimental_data.keys():
    dissimilarity = 1 - experimental_data[key]
    
    embedding = MDS(n_components=n_components, metric = True, dissimilarity = "precomputed", max_iter = 10000, eps = 1e-100)
    init = embedding.fit_transform(dissimilarity)

    embedding = MDS(n_components=n_components, metric = False, dissimilarity = "precomputed", max_iter = 10000, eps = 1e-100)
    transformed = embedding.fit_transform(dissimilarity, init = init)
    
    transformed_data[key] = transformed

# Plot MDS embedding

In [ ]:
annotations = list_of_chords

X = transformed_data["similarity"][:,0]
Y = transformed_data["similarity"][:,1]
Z = transformed_data["similarity"][:,2]


val = []
for l in list_of_chords:
    cdist = np.abs(np.diff(json.loads(l))[0])
    if cdist > 6:
        cdist = 12 - cdist
    val.append(cdist)
df = pd.DataFrame({"X": X, "Y": Y, "Z": Z, "cat": list_of_chords, "val": val})
fig = px.scatter_3d(df, x='X', y='Y', z='Z', text="cat", color='val', width=1000, height=1000,
                        title="MDS Space of Dyads")
fig.show()

# Plot transition difference against similarity

In [ ]:
import itertools
from scipy import stats

def compute_wrapped_distance(chord1, chord2, p): 
    diffs1 = np.mod(np.abs(np.array(chord1) - np.array(chord2)), 12)
    for i, diff in enumerate(diffs1):
        if diff > 6:
            diffs1[i] = 12 - diff
    diffs2 = np.mod(np.abs(np.flip(np.array(chord1)) - np.array(chord2)), 12)
    for i, diff in enumerate(diffs2):
        if diff > 6:
            diffs2[i] = 12 - diff
    diffs1 = diffs1**p
    diffs2 = diffs2**p
    diffs1 = np.power(diffs1, p)
    diffs2 = np.power(diffs2, p)
    dist = min(np.sum(diffs1), np.sum(diffs2))
    return dist ** (1/p)

def compute_distance(c1, c2, p):
    v = np.sum(np.abs(c1-c2)**p)
    return v ** 1/p 


chord_sim = experimental_data["similarity"]

dists = []
sims = []
P = 1
for i in range(len(list_of_chords)):
    for j in range(i+1,len(list_of_chords)):
        diff = compute_distance(np.array(json.loads(list_of_chords[i])), np.array(json.loads(list_of_chords[j])),P)
        sim = chord_sim[i,j]
        dists = dists + [diff]
        sims = sims + [sim]

fig, ax = plt.subplots(1,1,figsize = (7,7))
ax.scatter(dists,sims)
ax.set_xlabel("L1 distance", fontsize = 20)
ax.set_ylabel("Similarity", fontsize = 20)

print(stats.pearsonr(dists, sims))

mean_sim = []
cond_distance = []
for i in range(int(min(dists)), int(max(dists)) + 1):
    dvals = np.array([int(d) for d in dists])
    dsims = np.array(sims)
    mean_sim = mean_sim + [np.mean(dsims[dvals == i])]
    cond_distance = cond_distance + [i]

fig, ax = plt.subplots(1,1,figsize = (7,7))
ax.scatter(cond_distance,mean_sim)
ax.set_xlabel("L1 distance", fontsize = 20)
ax.set_ylabel("Mean Similarity", fontsize = 20)

dists = []
sims = []
P = 1
for i in range(len(list_of_chords)):
    for j in range(i+1,len(list_of_chords)):
        diff = compute_wrapped_distance(np.array(json.loads(list_of_chords[i])), np.array(json.loads(list_of_chords[j])), P)
        sim = chord_sim[i,j]
        dists = dists + [diff]
        sims = sims + [sim]

print(stats.pearsonr(dists, sims))

fig, ax = plt.subplots(1,1,figsize = (7,7))
ax.scatter(dists,sims)
ax.set_xlabel("Wrapped L1 distance", fontsize = 20)
ax.set_ylabel("Similarity", fontsize = 20)

mean_sim = []
sd_sim = []
cond_distance = []
for i in range(int(min(dists)), int(max(dists)) + 1):
    dvals = np.array([int(d) for d in dists])
    dsims = np.array(sims)
    mean_sim = mean_sim + [np.mean(dsims[dvals == i])]
    sd_sim = sd_sim + [np.std(dsims[dvals == i]) / np.sqrt(len(dsims[dvals == i]))] # std of mean
    cond_distance = cond_distance + [i]
    
fig, ax = plt.subplots(1,1,figsize = (7,7))
ax.scatter(cond_distance,mean_sim)
ax.errorbar(cond_distance,mean_sim,1.96*np.array(sd_sim))
ax.set_xlabel("Wrapped L1 distance", fontsize = 20)
ax.set_ylabel("Mean Similarity", fontsize = 20)

# Simulating similarity matrix based on wrapped L1 distance

In [ ]:
simulated_sim_mat = np.zeros((len(list_of_chords), len(list_of_chords)))
for i in range(len(list_of_chords)):
    for j in range(len(list_of_chords)):
        simulated_sim_mat[i,j] = compute_wrapped_distance(np.array(json.loads(list_of_chords[i])), np.array(json.loads(list_of_chords[j])), P)

fig, ax = plt.subplots(1,2,figsize = (12,12))
ax[0].matshow(1 - simulated_sim_mat / np.max(simulated_sim_mat))
ax[1].matshow(experimental_data["similarity"])

# Minimal Voice-leading Distance Analysis
Run a linear regression with the calculated wrapped distances and perceived similarities. Compare similarities predicted by the model with perceived distances.

In [ ]:
fit = scipy.stats.linregress(dists, sims)

fig, ax = plt.subplots(1,1,figsize = (7,7))

pred_sims = np.multiply(dists, fit[0]) + fit[1]
ax.plot(dists, pred_sims, color = "RED")
ax.scatter(dists, sims, alpha = 0.3)

dist_var = np.var(abs(pred_sims-sims))
ax.set_xlabel("Wrapped L1 distance", fontsize = 20)
ax.set_ylabel("Similarity", fontsize = 20)
print("Slope: ", fit[0], ", y-intercept: ", fit[1], "r-value: ", fit[2], "\n p-value: ", fit[3], "variance: ", dist_var)

# Spectral Pitch-class Distance Implementation
Implement spectral pitch-class distance function. Calculate distance for every pair of chords.

In [ ]:
# Spectral Pitch-class Distance
C = 60
harmonic_series = np.array([0, 1200, 700, 500, 400, 300, 300, 200, 200, 200, 200, 100])

# caclulates spectral distance between two chords
# rho = shrinking factor of successive harmonics
# sigma = variance of smeared distribution
def spectral_dist(chord1, chord2, rho, sigma) :
    # for each note, get row vector representation
    x_wt = np.zeros(1200)
    y_wt = np.zeros(1200)
    for i in range(2) :
        x_wt = x_wt + note_to_vec(chord1[i], rho)
        y_wt = y_wt + note_to_vec(chord2[i], rho)
        
    smear = np.random.normal(1, sigma, 1200)
    x_e = np.convolve(x_wt, smear,'same' )
    y_e = np.convolve(y_wt, smear, 'same')
    
    xy = x_e @ y_e
    xx = x_e @ x_e
    yy = y_e @ y_e
    
    return 1 - (xy / np.sqrt(xx * yy))
    
# converts single tone to frequency vector representation
def note_to_vec(note, rho) :
    idx = ((note - C) * 100) % 1200
    harm = np.zeros(1200)
    for i in range(len(harmonic_series)) :
        idx += harmonic_series[i]
        idx %= 1200
        harm[idx] = (i+1)**-rho
    return harm
        
# compute spectral distances
specs = []
rho = 0.6
sigma = 1
for i in range(len(list_of_chords)):
    for j in range(i+1, len(list_of_chords)):
        spec = spectral_dist(np.array(json.loads(list_of_chords[i])), np.array(json.loads(list_of_chords[j])), rho, sigma)
        specs = specs + [spec]





 # Spectral Optimization and Analysis
 Optimizing for the hyperparameters rho and sigma was not successful (commented out). Once again, run a linear regression with the calculated distances and perceived similarities. Compare similarities predicted by the model with perceived distances.

In [ ]:
b = scipy.optimize.Bounds(lb = [0.1,0.1], ub = [0.6, 1], keep_feasible = [True,True])  


def spectral_opt(params) :
    specs = []
    for i in range(len(list_of_chords)):
        for j in range(i+1, len(list_of_chords)):
            spec = spectral_dist(np.array(json.loads(list_of_chords[i])), np.array(json.loads(list_of_chords[j])),
                    params[0], params[1])
            specs = specs + [spec]
    fit = scipy.stats.linregress(specs, sims)
    spec_sims = np.multiply(specs, fit[0]) + fit[1]
    
    
    return np.linalg.norm(spec_sims-sims)

guess_params = [0.5, 0.9]
#result = minimize(spectral_opt, guess_params, bounds = b)

#print(result)



fig, ax = plt.subplots(1,1,figsize = (7,7))
ax.scatter(specs, sims, alpha = 0.2)

ax.set_xlabel("Spectral Pitch-class Distance", fontsize = 20)
ax.set_ylabel("Similarity", fontsize = 20)
print("Slope: ", fit[0], ", y-intercept: ", fit[1], "r-value: ", fit[2], 
      "\n p-value: ", fit[3], "variance: ", spec_var)


# Consonance Difference
Implement consonance difference function.

In [ ]:
col_names = ["answer","intervals"]
consonance_dyads = pd.read_csv(r"data/consonance_judgments_dyads.csv", usecols=col_names, low_memory=False)

# intervals are strings, answers are numpy.float64
consonance_dyads = consonance_dyads.groupby('intervals').mean().reset_index()


# given two dyads, find the difference in their consonance ratings
def cons_dif(chord1, chord2, cons) :
    cons1 = cons.loc[cons["intervals"]==chord1]["answer"].tolist()[0]
    cons2 = cons.loc[cons["intervals"]==chord2]["answer"].tolist()[0]
    return cons1-cons2

# compute differences for every combination of chords
diffs = []
for i in range(len(list_of_chords)):
    for j in range(i+1, len(list_of_chords)):
        diff = cons_dif(list_of_chords[i], list_of_chords[j], consonance_dyads)
        diffs = diffs +[diff]
        
        
# to get number of participants
part =  pd.read_csv(r"data/consonance_judgments_dyads.csv", usecols=['participant_id'], low_memory=False)


# Absolute Consonance Difference Analysis

In [ ]:
abs_diffs = np.abs(diffs)

fit = scipy.stats.linregress(abs_diffs, sims)


abs_diff_sims = np.multiply(abs_diffs, fit[0]) + fit[1]

cons_abs_var = np.var(abs(abs_diff_sims-sims))

fig, ax = plt.subplots(1,1,figsize = (7,7))
ax.scatter(abs_diffs, sims, alpha = 0.2)
ax.plot(abs_diffs, abs_diff_sims, color="RED")
ax.set_xlabel("Absolute Difference in Consonance Rating", fontsize = 20)
ax.set_ylabel("Similarity", fontsize = 20)
print("Slope: ", fit[0], ", y-intercept: ", fit[1], "r-value: ", fit[2], 
      "\n p-value: ", fit[3], "variance: ", cons_abs_var)

# Gaussian Consonance Difference Optimization and Analysis
Optimizing for sigma did not improve model performance.

In [ ]:
from scipy.optimize import minimize

# objective function to optimize sigma
def gauss_opt(sigma) :
    gaussian_diffs = np.exp(-np.power(diffs,2)) / sigma
    fit = scipy.stats.linregress(gaussian_diffs, sims)
    gsims = np.multiply(gaussian_diffs, fit[0]) + fit[1]
    return np.linalg.norm(gsims - sims)

guess_sigma = 1
result = minimize(gauss_opt, guess_sigma)

# Gaussian differences in consonance rating
best_gauss = np.exp(-np.power(diffs,2)) / result['x'][0]


fit = scipy.stats.linregress(best_gauss, sims)

# similarities predicted by optimized Gaussian difference
gauss_sims = np.multiply(best_gauss, fit[0]) + fit[1]

cons_gauss_var = np.var(abs(gauss_sims-sims))

fig, ax = plt.subplots(1,1,figsize = (7,7))
ax.scatter(best_gauss, sims, alpha = 0.2)
ax.plot(best_gauss, gauss_sims, color="RED")
ax.set_xlabel("Gaussian Difference in Consonance", fontsize = 20)
ax.set_ylabel("Similarity", fontsize = 20)
print("Slope: ", fit[0], ", y-intercept: ", fit[1], ", r-value: ", fit[2], 
      ",\n p-value: ", fit[3], ", variance: ", cons_gauss_var, ", optimized sigma:", result['x'][0])


# Sigmoid Consonance Difference Analysis
Unhelpful data transformation.

In [ ]:
sigmoid_diffs = 1 / (1 + np.exp(np.multiply(diffs,-1)))

fit = scipy.stats.linregress(sigmoid_diffs, sims)

diff_sims = np.multiply(sigmoid_diffs, fit[0]) + fit[1]

cons_sig_var = np.var(abs(diff_sims-sims))
fig, ax = plt.subplots(1,1,figsize = (7,7))
ax.scatter(sigmoid_diffs, sims, alpha = 0.2)
ax.plot(sigmoid_diffs, diff_sims, color="RED")
ax.set_xlabel("Sigmoid Difference in Consonance Rating", fontsize = 20)
ax.set_ylabel("Similarity", fontsize = 20)
print("Slope: ", fit[0], ", y-intercept: ", fit[1], ", r-value: ", fit[2], 
      ",\n p-value: ", fit[3], ", variance: ", cons_sig_var)


# Combined Model Analysis
Linear regression of two variables using minimal voice-leading distance and Gaussian consonance difference data as predictors for perceived similarity data.

In [ ]:

from sklearn import linear_model
mixture = linear_model.LinearRegression()
# use Gaussian differences
mixture.fit(np.array([dists,best_gauss]).T, sims)

mix_pred_sims = np.multiply(dists,mixture.coef_[0]) + np.multiply(best_gauss,mixture.coef_[1]) + mixture.intercept_
mix_r = np.sqrt(mixture.score(np.array([dists,best_gauss]).T, sims))
mix_var = np.var(abs(mix_pred_sims-sims))

chord_pairs = []
for i in range(len(list_of_chords)):
    for j in range(i+1, len(list_of_chords)):
        pair = [list_of_chords[i], list_of_chords[j]]
        chord_pairs = chord_pairs + [pair]


mix_df = pd.DataFrame({"Wrapped Distance" : dists, "Consonance Difference" : best_gauss, "Pair" : chord_pairs, 
                       "Similarity" : sims})
fig = px.scatter_3d(mix_df, x='Wrapped Distance', y='Consonance Difference', 
                    z='Similarity', color = 'Similarity', width=1000, 
                    height=1000, title='Multivariate Fit' )
fig.show()


print("Distance contribution: ", mixture.coef_[0], ", Gaussian Consonance Difference contribution: ", 
      mixture.coef_[1], ", \n r: ", mix_r, ", variance: ", mix_var)


In [ ]:
fig, ax = plt.subplots(1,1,figsize = (7,7))
ax.scatter(dists, best_gauss,c=mix_pred_sims,  alpha = 0.8)
ax.set_xlabel("Wrapped Distance", fontsize = 20)
ax.set_ylabel("Gaussian Consonance Difference", fontsize = 20)


# Standard Voice-leading Distance Analysis
As a baseline

In [ ]:
chord_sim = experimental_data["similarity"]

dists = []
sims = []
P = 1
for i in range(len(list_of_chords)):
    for j in range(i+1,len(list_of_chords)):
        diff = compute_distance(np.array(json.loads(list_of_chords[i])), np.array(json.loads(list_of_chords[j])),P)
        sim = chord_sim[i,j]
        dists = dists + [diff]
        sims = sims + [sim]
        
fit = scipy.stats.linregress(dists, sims)

st_sims = np.multiply(dists, fit[0]) + fit[1]

st_var = np.var(abs(st_sims-sims))

fig, ax = plt.subplots(1,1,figsize = (7,7))
ax.scatter(dists, sims, alpha = 0.2)
ax.plot(dists, st_sims, color="RED")
ax.set_xlabel("L1 Distance", fontsize = 20)
ax.set_ylabel("Similarity", fontsize = 20)
print("Slope: ", fit[0], ", y-intercept: ", fit[1], ", r-value: ", fit[2], 
      ",\n p-value: ", fit[3], ", variance: ", st_var)